Mounting Drive Folders



In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Install requirement.

In [ ]:
!pip install git+https://github.com/FacePerceiver/facer.git@main
!git clone https://github.com/FacePerceiver/facer.git

  Cloning https://github.com/FacePerceiver/facer.git (to revision main) to /tmp/pip-req-build-1fo7py7d
  Running command git clone --filter=blob:none --quiet https://github.com/FacePerceiver/facer.git /tmp/pip-req-build-1fo7py7d
  Encountered 16 file(s) that should have been pointers, but weren't:
        samples/data/fire.webp
        samples/data/girl.jpg
        samples/data/sideface.jpg
        samples/data/twogirls.jpg
        samples/data/weirdface.jpg
        samples/data/weirdface2.jpg
        samples/data/weirdface3.jpg
        samples/download.ipynb
        samples/example_output/alignment.png
        samples/example_output/detect.png
        samples/example_output/parsing.png
        samples/face_alignment.ipynb
        samples/face_attribute.ipynb
        samples/face_detect.ipynb
        samples/face_parsing.ipynb
        samples/transform.ipynb
  Resolved https://github.com/FacePerceiver/facer.git to commit f0dd2d67def16445454b90a191cc763431795979
  Preparing metadata (se

Face Parsing for single image

In [ ]:
import torch
import cv2
import numpy as np
import facer

def save_skin_mask(img_path, output_path="temp.jpg"):
    # Using CUDA GPU if available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Read and pre-process the image for Facer
    sample = cv2.imread(img_path)
    if sample is None:
        print(f"Error: Unable to read image at {img_path}")
        return
    img = cv2.cvtColor(sample, cv2.COLOR_BGR2RGB)

    # Convert img (numpy array) to PyTorch tensor
    image = torch.from_numpy(img).to(device=device)

    # Change the order of the dimensions from HWC to CHW
    image = image.permute(2, 0, 1).unsqueeze(0)

    # Load Facer models
    face_detector = facer.face_detector('retinaface/mobilenet', device=device)
    face_parser = facer.face_parser('farl/lapa/448', device=device)

    with torch.inference_mode():
        faces = face_detector(image)
        if faces['rects'].nelement() == 0:
            print(f"No faces detected in {img_path}, skipping...")
            return

        if 'image_ids' in faces:
            faces['image_ids'] = faces['image_ids'].long()
        faces = face_parser(image, faces)

    # Parse the face skin
    seg_logits = faces['seg']['logits']
    seg_probs = seg_logits.softmax(dim=1).cpu()  # Ensure CPU for numpy compatibility
    tensor = seg_probs.permute(0, 2, 3, 1).squeeze().numpy()
    face_skin = tensor[:, :, 1]

    # Create binary mask
    binary_mask = (face_skin >= 0.5).astype(np.uint8)

    # Debug mask dimensions
    if binary_mask.ndim != 2:
        print(f"Mask dimension issue in {img_path}: {binary_mask.shape}")
        return

    # Resize binary mask to match the original image
    binary_mask = cv2.resize(binary_mask, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Create masked image
    masked_image = np.zeros_like(img)
    try:
        masked_image[binary_mask == 1] = img[binary_mask == 1]
        masked_image = cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB)
        cv2.imwrite(output_path, masked_image)
        print(f"Masked image saved to {output_path}")
    except Exception as e:
        print(f"Error occurred while creating the masked image: {e}")

# Example Usage
save_skin_mask("./facer/samples/data/girl.jpg", "output_masked.jpg")

Masked image saved to output_masked.jpg


Face Parsing for dataset folder

In [ ]:
import os
import cv2
import numpy as np
import torch
import facer

# Paths
base_path = '/content/drive/MyDrive/Deep_Armocromia/'
data_types = ['train', 'test']
output_folder = '/content/drive/MyDrive/extracted_skin'

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load facer models
face_detector = facer.face_detector('retinaface/mobilenet', device=device)
face_parser = facer.face_parser('farl/lapa/448', device=device)

# Process images
for data_type in data_types:
    data_path = os.path.join(base_path, data_type)
    output_data_path = os.path.join(output_folder, data_type)
    os.makedirs(output_data_path, exist_ok=True)

    # Iterate through seasons (autumn, spring, etc.)
    for season_name in os.listdir(data_path):
        season_path = os.path.join(data_path, season_name)
        output_season_path = os.path.join(output_data_path, season_name)
        os.makedirs(output_season_path, exist_ok=True)

        # Iterate through sub-seasons (deep autumn, etc.)
        for sub_season_name in os.listdir(season_path):
            sub_season_path = os.path.join(season_path, sub_season_name)
            output_sub_season_path = os.path.join(output_season_path, sub_season_name)
            os.makedirs(output_sub_season_path, exist_ok=True)

            if os.path.isdir(sub_season_path):
                for filename in os.listdir(sub_season_path):
                    if filename.endswith(('.jpg', '.png')):
                        sample_path = os.path.join(sub_season_path, filename)
                        sample = cv2.imread(sample_path)

                        # Check if image is loaded properly
                        if sample is None:
                            print(f"Failed to load image: {sample_path}")
                            continue

                        img = cv2.cvtColor(sample, cv2.COLOR_BGR2RGB)

                        # Read and preprocess the image for facer
                        image = facer.hwc2bchw(facer.read_hwc(sample_path)).to(device=device)
                        with torch.inference_mode():
                            faces = face_detector(image)
                            if faces['rects'].nelement() == 0:
                                print(f"No faces detected in {filename}, skipping...")
                                continue
                            if 'image_ids' in faces:
                                faces['image_ids'] = faces['image_ids'].long()
                            faces = face_parser(image, faces)

                        # Parse the face skin
                        seg_logits = faces['seg']['logits']
                        seg_probs = seg_logits.softmax(dim=1).cpu()
                        tensor = seg_probs.permute(0, 2, 3, 1).squeeze().numpy()
                        face_skin = tensor[:, :, 1]

                        # Create binary mask
                        binary_mask = (face_skin >= 0.5).astype(int)

                        # Debug mask dimensions
                        if binary_mask.ndim != 2:
                            print(f"Mask dimension issue in {filename}: {binary_mask.shape}")
                            continue

                        # Resize binary mask to match image
                        # binary_mask = cv2.resize(binary_mask, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)

                        # Create masked image
                        masked_image = np.zeros_like(img)
                        masked_image[binary_mask == 1] = img[binary_mask == 1]

                        # Convert to BGR for saving
                        masked_image_rgb = cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB)

                        # Save masked image
                        output_image_path = os.path.join(output_sub_season_path, filename)
                        success = cv2.imwrite(output_image_path, masked_image_rgb)
                        if success:
                            print(f"Saved masked image to: {output_image_path}")
                        else:
                            print(f"Failed to save masked image: {output_image_path}")

/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Saved masked image to: /content/drive/MyDrive/extracted_skin/train/winter/clear winter/laura graham 1.png
Saved masked image to: /content/drive/MyDrive/extracted_skin/train/winter/clear winter/paola turani44.png
Saved masked image to: /content/drive/MyDrive/extracted_skin/train/winter/clear winter/Katy Perry__1.png
Mask dimension issue in paola turani52.png: (2, 694, 11)
Saved masked image to: /content/drive/MyDrive/extracted_skin/train/winter/clear winter/paola turani63.png
Saved masked image to: /content/drive/MyDrive/extracted_skin/train/winter/clear winter/paola turani49.png
Saved masked image to: /content/drive/MyDrive/extracted_skin/train/winter/clear winter/paola turani53.png
Saved masked image to: /content/drive/MyDrive/extracted_skin/train/winter/clear winter/alice-basso1.png
Saved masked image to: /content/drive/MyDrive/extracted_skin/train/winter/clear winter/raoul bova 2.png
Saved masked image to: /content/drive/MyDrive/extracted_skin/train/winter/clear winter/paola turani4

cropped to only face mask size

In [ ]:
import os
import cv2
import numpy as np
import torch
import facer

# Paths
base_path = '/content/drive/MyDrive/extracted_face-skin/'
data_types = ['train', 'test']
output_folder = '/content/drive/MyDrive/skin_only_images'

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load facer models
face_detector = facer.face_detector('retinaface/mobilenet', device=device)
face_parser = facer.face_parser('farl/lapa/448', device=device)

# Process images
for data_type in data_types:
    data_path = os.path.join(base_path, data_type)
    output_data_path = os.path.join(output_folder, data_type)
    os.makedirs(output_data_path, exist_ok=True)

    for folder_name in os.listdir(data_path):
        folder_path = os.path.join(data_path, folder_name)
        output_folder_path = os.path.join(output_data_path, folder_name)
        os.makedirs(output_folder_path, exist_ok=True)

        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                if filename.endswith(('.jpg', '.png')):
                    sample_path = os.path.join(folder_path, filename)
                    sample = cv2.imread(sample_path)

                    # Check if image is loaded properly
                    if sample is None:
                        print(f"Failed to load image: {sample_path}")
                        continue

                    img = cv2.cvtColor(sample, cv2.COLOR_BGR2RGB)

                    # Read and preprocess the image for facer
                    image = facer.hwc2bchw(facer.read_hwc(sample_path)).to(device=device)
                    with torch.inference_mode():
                        faces = face_detector(image)
                        if faces['rects'].nelement() == 0:
                            print(f"No faces detected in {filename}, skipping...")
                            continue
                        if 'image_ids' in faces:
                            faces['image_ids'] = faces['image_ids'].long()
                        faces = face_parser(image, faces)

                    # Parse the face skin
                    seg_logits = faces['seg']['logits']
                    seg_probs = seg_logits.softmax(dim=1).cpu()
                    tensor = seg_probs.permute(0, 2, 3, 1).squeeze().numpy()
                    face_skin = tensor[:, :, 1]

                    # Create binary mask
                    binary_mask = (face_skin >= 0.5).astype(int)

                    # Debug mask dimensions
                    if binary_mask.ndim != 2:
                        print(f"Mask dimension issue in {filename}: {binary_mask.shape}")
                        continue

                    # Resize binary mask to match image
                    binary_mask = cv2.resize(binary_mask, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)

                    # Extract bounding box of the face skin
                    y_indices, x_indices = np.where(binary_mask == 1)
                    if len(y_indices) == 0 or len(x_indices) == 0:
                        print(f"No skin pixels found in {filename}, skipping...")
                        continue

                    # Calculate bounding box
                    x_min, x_max = x_indices.min(), x_indices.max()
                    y_min, y_max = y_indices.min(), y_indices.max()

                    # Crop the image to the bounding box
                    cropped_face_skin = img[y_min:y_max + 1, x_min:x_max + 1]

                    # Convert to BGR for saving
                    cropped_face_skin_bgr = cv2.cvtColor(cropped_face_skin, cv2.COLOR_BGR2RGB)

                    # Save the cropped image
                    output_image_path = os.path.join(output_folder_path, filename)
                    success = cv2.imwrite(output_image_path, cropped_face_skin_bgr)
                    if success:
                        print(f"Saved cropped face skin image to: {output_image_path}")
                    else:
                        print(f"Failed to save cropped face skin image: {output_image_path}")


Saved cropped face skin image to: /content/drive/MyDrive/skin_only_images/train/cool winter/2125.png
Saved cropped face skin image to: /content/drive/MyDrive/skin_only_images/train/cool winter/6104.png
Saved cropped face skin image to: /content/drive/MyDrive/skin_only_images/train/cool winter/SHAKIRA 1.png
Saved cropped face skin image to: /content/drive/MyDrive/skin_only_images/train/cool winter/16517.png
Saved cropped face skin image to: /content/drive/MyDrive/skin_only_images/train/cool winter/Irene Colzi.png
Saved cropped face skin image to: /content/drive/MyDrive/skin_only_images/train/cool winter/5869.png
Saved cropped face skin image to: /content/drive/MyDrive/skin_only_images/train/cool winter/16071.png
Saved cropped face skin image to: /content/drive/MyDrive/skin_only_images/train/cool winter/anne hathaway21.png
Saved cropped face skin image to: /content/drive/MyDrive/skin_only_images/train/cool winter/7401.png
Saved cropped face skin image to: /content/drive/MyDrive/skin_only

saving while renaming the file

In [ ]:
import os
import cv2
import numpy as np
import torch
import facer
from PIL import Image

# Paths
base_path = '/content/drive/MyDrive/masked_images_face/'
data_types = ['train', 'test']
output_folder = '/content/drive/MyDrive/skin_only_images'

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Device configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load facer models
face_detector = facer.face_detector('retinaface/mobilenet', device=device)
face_parser = facer.face_parser('farl/lapa/448', device=device)

def process_image(image_path, device):
    """Process a single image to extract the face skin region."""
    # Load and check image
    sample = cv2.imread(image_path)
    if sample is None:
        print(f"Failed to load image: {image_path}")
        return None

    img = cv2.cvtColor(sample, cv2.COLOR_BGR2RGB)

    # Read and preprocess the image for facer
    image = facer.hwc2bchw(facer.read_hwc(image_path)).to(device)
    with torch.inference_mode():
        faces = face_detector(image)
        if faces['rects'].nelement() == 0:
            print(f"No faces detected in {os.path.basename(image_path)}, skipping...")
            return None

        faces = face_parser(image, faces)

    # Parse the face skin
    seg_logits = faces['seg']['logits']
    seg_probs = seg_logits.softmax(dim=1).cpu()
    tensor = seg_probs.permute(0, 2, 3, 1).squeeze().numpy()
    face_skin = tensor[:, :, 1]

    # Create binary mask
    binary_mask = (face_skin >= 0.5).astype(int)
    return img, binary_mask

def save_cropped_image(img, mask, output_path, make_transparent=True):
    """
    Save the cropped image with or without transparency.
    """
    try:
        # Ensure mask and image dimensions match
        if make_transparent:
            if mask.shape[:2] != img.shape[:2]:
                mask = cv2.resize(mask, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)

            # Create an RGBA image
            rgba_image = np.zeros((img.shape[0], img.shape[1], 4), dtype=np.uint8)
            rgba_image[:, :, :3] = img  # Copy RGB channels
            rgba_image[:, :, 3] = (mask * 255).astype(np.uint8)  # Alpha channel

            # Save as PNG
            output_image = Image.fromarray(rgba_image, 'RGBA')
            output_image.save(output_path, format='PNG')
        else:
            # Save as a standard RGB image
            output_image = Image.fromarray(img, 'RGB')
            output_image.save(output_path, format='JPEG')

        print(f"Saved image to: {output_path}")
        return True

    except Exception as e:
        print(f"Error saving image to {output_path}: {e}")
        return False


# Process images
for data_type in data_types:
    data_path = os.path.join(base_path, data_type)
    output_data_path = os.path.join(output_folder, data_type)
    os.makedirs(output_data_path, exist_ok=True)

    for folder_name in os.listdir(data_path):
        folder_path = os.path.join(data_path, folder_name)
        output_folder_path = os.path.join(output_data_path, folder_name)
        os.makedirs(output_folder_path, exist_ok=True)

        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                if filename.endswith(('.jpg', '.png')):
                    sample_path = os.path.join(folder_path, filename)
                    output_image_path = os.path.join(output_folder_path, filename.split('.')[0] + '.png')

                    # Process the image
                    result = process_image(sample_path, device)
                    if result:
                        img, mask = result
                        save_cropped_image(img, mask, output_image_path, make_transparent=True)


Saved image to: /content/drive/MyDrive/skin_only_images/train/light spring/light_spring_1.png
Saved image to: /content/drive/MyDrive/skin_only_images/train/light spring/light_spring_2.png
Saved image to: /content/drive/MyDrive/skin_only_images/train/light spring/light_spring_3.png
Saved image to: /content/drive/MyDrive/skin_only_images/train/light spring/light_spring_4.png
Saved image to: /content/drive/MyDrive/skin_only_images/train/light spring/light_spring_5.png
Saved image to: /content/drive/MyDrive/skin_only_images/train/light spring/light_spring_6.png
Saved image to: /content/drive/MyDrive/skin_only_images/train/light spring/light_spring_7.png
No faces detected in light_spring_8.png, skipping...
Saved image to: /content/drive/MyDrive/skin_only_images/train/light spring/light_spring_9.png
Saved image to: /content/drive/MyDrive/skin_only_images/train/light spring/light_spring_10.png
Saved image to: /content/drive/MyDrive/skin_only_images/train/light spring/light_spring_11.png
Saved

Create CSV file

In [ ]:
# importing csv
import csv

# Data
data = [
    ['hex', 'season'],
    ['#f6a390', 'clear spring'],
    ['#ff8d6d', 'clear spring'],
    ['#ed5475', 'clear spring'],
    ['#ff565b', 'clear spring'],
    ['#db2a1c', 'clear spring'],
    ['#ef61a4', 'clear spring'],
    ['#d438b4', 'clear spring'],
    ['#973cbe', 'clear spring'],
    ['#af98de', 'clear spring'],
    ['#463536', 'clear spring'],
    ['#9393d1', 'warm spring'],
    ['#973cbd', 'warm spring'],
    ['#d9281a', 'warm spring'],
    ['#b36953', 'warm spring'],
    ['#ed5138', 'warm spring'],
    ['#a5621a', 'warm spring'],
    ['#ff8000', 'warm spring'],
    ['#ffa38b', 'warm spring'],
    ['#fab57c', 'warm spring'],
    ['#ffba1c', 'warm spring'],
    ['#faac8f', 'light spring'],
    ['#fdad66', 'light spring'],
    ['#ff8d6d', 'light spring'],
    ['#fcc89a', 'light spring'],
    ['#c4b2e3', 'light spring'],
    ['#d7a8e2', 'light spring'],
    ['#f7cda7', 'light spring'],
    ['#c2a695', 'light spring'],
    ['#cfa279', 'light spring'],
    ['#9696d2', 'light spring'],
    ['#f395c7', 'clear winter'],
    ['#89189f', 'clear winter'],
    ['#a61891', 'clear winter'],
    ['#ab0062', 'clear winter'],
    ['#c5007d', 'clear winter'],
    ['#e20888', 'clear winter'],
    ['#e40047', 'clear winter'],
    ['#cb0033', 'clear winter'],
    ['#c824b2', 'clear winter'],
    ['#9c40c2', 'clear winter'],
    ['#f192c5', 'cool winter'],
    ['#83319a', 'cool winter'],
    ['#933bbc', 'cool winter'],
    ['#c622af', 'cool winter'],
    ['#db1886', 'cool winter'],
    ['#e2006b', 'cool winter'],
    ['#cf0038', 'cool winter'],
    ['#a7005e', 'cool winter'],
    ['#8a0c59', 'cool winter'],
    ['#83309a', 'deep winter'],
    ['#963cbd', 'deep winter'],
    ['#c723b0', 'deep winter'],
    ['#da1784', 'deep winter'],
    ['#e50072', 'deep winter'],
    ['#d00039', 'deep winter'],
    ['#a7005e', 'deep winter'],
    ['#991e68', 'deep winter'],
    ['#8b0b5a', 'deep winter'],
    ['#f0bbc6', 'soft summer'],
    ['#ffa3b5', 'soft summer'],
    ['#f65073', 'soft summer'],
    ['#d694ac', 'soft summer'],
    ['#a075a4', 'soft summer'],
    ['#a462b9', 'soft summer'],
    ['#a65e9a', 'soft summer'],
    ['#8a687e', 'soft summer'],
    ['#78658f', 'soft summer'],
    ['#a73b70', 'soft summer'],
    ['#f395c7', 'cool summer'],
    ['#f57eb6', 'cool summer'],
    ['#e174cc', 'cool summer'],
    ['#c660cd', 'cool summer'],
    ['#953bbc', 'cool summer'],
    ['#973392', 'cool summer'],
    ['#ae145b', 'cool summer'],
    ['#c00d3e', 'cool summer'],
    ['#e31c7a', 'cool summer'],
    ['#ee5c9f', 'cool summer'],
    ['#ecb3cb', 'light summer'],
    ['#de9ddf', 'light summer'],
    ['#f57195', 'light summer'],
    ['#f47ab3', 'light summer'],
    ['#cb67d1', 'light summer'],
    ['#a260b7', 'light summer'],
    ['#af98dd', 'light summer'],
    ['#9999d4', 'light summer'],
    ['#ccb3a5', 'light summer'],
    ['#dc874f', 'soft autumn'],
    ['#c26e5f', 'soft autumn'],
    ['#996f5b', 'soft autumn'],
    ['#935f36', 'soft autumn'],
    ['#bf502f', 'soft autumn'],
    ['#7a4b38', 'soft autumn'],
    ['#554942', 'soft autumn'],
    ['#b68251', 'soft autumn'],
    ['#a47800', 'warm autumn'],
    ['#a7641b', 'warm autumn'],
    ['#946037', 'warm autumn'],
    ['#9b4514', 'warm autumn'],
    ['#7c4d3a', 'warm autumn'],
    ['#a03626', 'warm autumn'],
    ['#c3602b', 'warm autumn'],
    ['#ec5037', 'warm autumn'],
    ['#ee7000', 'warm autumn'],
    ['#f68f2f', 'warm autumn'],
    ['#5e068f', 'deep autumn'],
    ['#81272b', 'deep autumn'],
    ['#643335', 'deep autumn'],
    ['#880b57', 'deep autumn'],
    ['#973cbe', 'deep autumn'],
    ['#9d3526', 'deep autumn'],
    ['#7b4c39', 'deep autumn'],
    ['#ec4d35', 'deep autumn'],
    ['#bd4c00', 'deep autumn'],
    ['#ff8300', 'deep autumn'],
    ['#c5622d', 'deep autumn']
]

# File path for the CSV file
csv_file_path = 'colors.csv'

# Open the file in write mode
with open(csv_file_path, mode='w', newline='') as file:
    # Create a csv.writer object
    writer = csv.writer(file)
    # Write data to the CSV file
    writer.writerows(data)

# Print a confirmation message
print(f"CSV file '{csv_file_path}' created successfully.")

CSV file 'colors.csv' created successfully.
